In [1]:
# !pip install quandl

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split   

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
df = quandl.get("WIKI/FB")
print(df.head())    

             Open   High    Low    Close       Volume  Ex-Dividend  \
Date                                                                 
2012-05-18  42.05  45.00  38.00  38.2318  573576400.0          0.0   
2012-05-21  36.53  36.66  33.00  34.0300  168192700.0          0.0   
2012-05-22  32.61  33.59  30.94  31.0000  101786600.0          0.0   
2012-05-23  31.37  32.50  31.36  32.0000   73600000.0          0.0   
2012-05-24  32.95  33.21  31.77  33.0300   50237200.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2012-05-18          1.0      42.05      45.00     38.00     38.2318   
2012-05-21          1.0      36.53      36.66     33.00     34.0300   
2012-05-22          1.0      32.61      33.59     30.94     31.0000   
2012-05-23          1.0      31.37      32.50     31.36     32.0000   
2012-05-24          1.0      32.95      33.21     31.77     33.0300   

           

In [6]:
df = df[['Adj. Close']]
df.head()

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300


In [7]:
#A variable for predicting 'n' days in future
forecast_out = 30
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
print(df.tail())  

            Adj. Close  Prediction
Date                              
2018-03-21      169.39         NaN
2018-03-22      164.89         NaN
2018-03-23      159.39         NaN
2018-03-26      160.06         NaN
2018-03-27      152.19         NaN


In [8]:
# CREATE AN INDEPENDENT DATA SET (X)
#Convert Data Frame to numpy array
x = np.array(df.drop(['Prediction'],1))
x = x[:-forecast_out]
print(x)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


C:\Users\pantv\AppData\Local\Temp\ipykernel_13872\1888340487.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(df.drop(['Prediction'],1))


In [9]:
y = np.array(df['Prediction'])
y = y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [10]:
# SPLIT THE DATA TO 80% TRAINING AND 20% TESTING

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [11]:
#CREATE AND TRAIN THE SUPPORT VECTOR MACHINE(REGRESSOR)
svr_rbf = SVR(kernel = 'rbf', C = 1e3, gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, gamma=0.1)

In [12]:
#Testing the Model : Score returns the coefficient of determination R^2 of the prediction
#The best possible score is 1.0
svm_confindence = svr_rbf.score(x_test,y_test)
print("svm_confidence: ",svm_confindence)

svm_confidence:  0.9861953392844058


In [13]:
#Create and Train an Linear Regression Model
lr = LinearRegression()
#Train the model
lr.fit(x_train,y_train)

LinearRegression()

In [14]:
#Testing the Model : Score returns the coefficient of determination R^2 of the prediction
#The best possible score is 1.0

lr_confindence = lr.score(x_test,y_test)
print("lr_confidence: ",lr_confindence)

lr_confidence:  0.9816558724046561


In [15]:
#Set x_forecast equal to last "forecast_out" rows of the original data set from Adj_close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)    

[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


C:\Users\pantv\AppData\Local\Temp\ipykernel_13872\4269522940.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]


In [17]:
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[176.97355176 183.40467481 183.84889681 181.22394863 179.86099476
 181.77922613 182.86958922 187.21084968 188.86658623 185.36329
 182.19316027 179.79032308 180.47684799 184.29311882 183.66716963
 187.63487977 186.251734   189.16946487 188.694955   185.78732009
 188.11948559 187.78631909 189.0281215  176.37789044 171.92557448
 173.17747285 168.6342933  163.08151829 163.75794724 155.81243101]


In [18]:
svr_prediction = svr_rbf.predict(x_forecast)
print(svr_prediction)

[175.57405778 181.08771261 178.78314121 179.54496799 174.53994427
 182.6472181  183.61257874 185.58879552 175.63039612 181.73596316
 184.09839164 174.49249325 175.82933111 177.52578198 179.66202793
 182.26357794 187.42923021 176.11610345 175.76697118 184.90890583
 178.37425351 180.98060802 175.7797946  176.07362359 171.15358599
 172.78752292 172.364684   167.79876538 169.8386244  159.91573722]
